In [1]:
from slackclient import SlackClient
import time
import os
import re
from time import strftime, gmtime, localtime
from datetime import datetime
import requests
import nltk
from nltk.stem.lancaster import LancasterStemmer
nltk.download('punkt')
import json
import numpy as np
from random import randint
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to /Users/anon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Slack bot setting up and starter code: https://github.com/mattmakai/slack-starterbot/blob/master/starterbot.py
%env SLACK_BOT_TOKEN='SLACK_BOT_TOKEN'
# instantiate Slack client
slack_client = SlackClient(os.environ.get('SLACK_BOT_TOKEN'))
# starterbot's user ID in Slack: value is assigned after the bot starts up
starterbot_id = None

# constants
RTM_READ_DELAY = 1 # 1 second delay between reading from RTM
EXAMPLE_COMMAND = "do"
MENTION_REGEX = "^<@(|[WU].+?)>(.*)"



#print(SLACK_BOT_TOKEN)

env: SLACK_BOT_TOKEN='SLACK_BOT_TOKEN'


In [10]:
if __name__ == "__main__":
    if slack_client.rtm_connect(with_team_state=False):
        print("Starter Bot connected and running!")
        # Read bot's user ID by calling Web API method `auth.test`
        starterbot_id = slack_client.api_call("auth.test")["user_id"]
        
        while True:
            command, channel = parse_bot_commands(slack_client.rtm_read())
            if command:
                handle_command(command, channel)
            time.sleep(RTM_READ_DELAY)
    else:
        print("Connection failed. Exception traceback printed above.")


Connection failed. Exception traceback printed above.


Traceback (most recent call last):
  File "/Users/anon/SlackBot/slackbot1/lib/python3.6/site-packages/slackclient/client.py", line 52, in rtm_connect
    self.server.rtm_connect(use_rtm_start=with_team_state, **kwargs)
  File "/Users/anon/SlackBot/slackbot1/lib/python3.6/site-packages/slackclient/server.py", line 151, in rtm_connect
    raise SlackLoginError(reply=reply)
slackclient.server.SlackLoginError


In [3]:
with open('pattern_response.json') as json_data:
    conversation = json.load(json_data)

In [4]:
# Text classification code - https://github.com/ugik/notebooks/blob/master/Neural_Network_Classifier.ipynb
words = []
documents = []
classes = []
responses = []
ignore_words = ['?']

#Cycling through each item - representing a tag-pattern-response in the json document)
for tags in conversation['converse_intents']:

#cycling through each sentence in a pattern (one pattern is a group of sentences) in the 
#item extracted in the first loop
    
    for patterns in tags['patterns']:

#tokenizing the sentences in each pattern - basically splits into individual words
        w = nltk.word_tokenize(patterns)

#adding (extending) to the words array
        words.extend(w)
        
#Combining the tokenized word(s) and the class it belongs to - which is represented by the 'tag' in the json doc 
#Useful to know the difference between append and extend
        documents.append((w, tags['tag']))

#Adding each of these 'tag' to the classes array - this is done outside of the inner loop to prevent duplication
#as there can be multiple patterns for each tag
    if tags['tag'] not in classes:
        classes.append(tags['tag'])
    
    if tags['responses'] not in responses:
        responses.append(tags['responses'])


In [5]:
#The words are stemmed - basically trimming to their roots - sometimes does not make much sense

words = [stemmer.stem(word.lower()) for word in words if word not in ignore_words]

#set- uniques the list of words generated
words = list(set(words))

#print(len(words))
#print(len(classes))
#print(len(documents))
#print(words)
#print(documents)

In [6]:
#initializing some arrays
binary_array = []
output_data = []
output_empty = [0]*len(classes)

#cycling through each of the 27 documents - a document is a tokenized pattern combined with its class
#enumerate helps in iterating a counter for each cycle of the loop

for i,item in enumerate(documents):
    
#extracts only the tokenized pattern leaving out the class    
    sentence = item[0]

#Now stemming the words in this tokenized pattern     
    unit_words = [stemmer.stem(word.lower()) for word in sentence]
    bag = []
    

#Now taking each of the word from the unique stemmed words done earlier and checking if it is in the stemmed tokenized 
#pattern extracted in the outer loop. if present, the bag array is appended by a 1 or a 0. Basically, we are taking 
# the uniques stemmed words made earlier and checking every time with the stemmed tokenized pattern and producing a 1 
# for a match

    for string in words: 
        if (string in unit_words):
            bag.append(1)
            
        else:
            bag.append(0)

#the second part of the document which is the class is taken and its index is found from the classes list created
#earlier and that particular row element is given a 1. For instance if the class index is 8, then the 8th element in 
# row is assigned 1. And a final array has the combination of the bag and the output.

    output_row = list(output_empty)
    output_row[classes.index(item[1])]=1
    binary_array.append(bag)
    output_data.append(output_row)

input_array = np.array(binary_array)
output_array = np.array(output_data)

In [7]:
def handling_input_text(input):
    ignore_words = ['?']
    input_bag = []
    
    input_token = nltk.word_tokenize(input)
    input_stem = [stemmer.stem(word.lower()) for word in input_token if word not in ignore_words]
    
    for string in words:
        if string in input_stem:
            input_bag.append(1)
        else:
            input_bag.append(0)
    
    input_bag = np.array(input_bag)
    match = np.dot(input_array,input_bag)
    
    match_class = match.argmax(axis=0)
    
    response_string = handling_response(output_array[match_class].argmax(axis=0))
    return(response_string)

In [8]:
def handling_response(class_index):
    response_counter = [0] * len(classes)
    nthresponse = randint(0,(len(responses[class_index])-1))
    output_response = responses[class_index][nthresponse]
    return(output_response)

In [9]:
def parse_bot_commands(slack_events):
    """
        Parses a list of events coming from the Slack RTM API to find bot commands.
        If a bot command is found, this function returns a tuple of command and channel.
        If its not found, then this function returns None, None.
    """
    for event in slack_events:
        if event["type"] == "message" and not "subtype" in event:
            user_id, message = parse_direct_mention(event["text"])
            if user_id == starterbot_id:
                return message, event["channel"]
    return None, None

def parse_direct_mention(message_text):
    """
        Finds a direct mention (a mention that is at the beginning) in message text
        and returns the user ID which was mentioned. If there is no direct mention, returns None
    """
    matches = re.search(MENTION_REGEX, message_text)
    # the first group contains the username, the second group contains the remaining message
    return (matches.group(1), matches.group(2).strip()) if matches else (None, None)

def handle_command(command, channel):
    """
        Executes bot command if the command is known
    """
    
    # Default response is help text for the user
    default_response = "Not sure what you mean. Try *{}*.".format(EXAMPLE_COMMAND)

    # Finds and executes the given command, filling in response
    response = None
    # This is where you start to implement more commands!
    
    #if command.startswith(EXAMPLE_COMMAND):
     #   response = "Sure...write some more code then I can do that!"
    
    time_check = re.compile('time')
    weather_check = re.compile('weather')
    if time_check.search(command):
        response = "The time now is %s" %strftime("%Y-%m-%d %H:%M:%S", localtime())
    elif weather_check.search(command):
        response = requests.get("http://api.openweathermap.org/data/2.5/weather?id=1880252&APPID=e58f9485573ee17e2ecff9a313420bcf").json()
    else:
        response = handling_input_text(command)
         
    
    
    # Sends the response back to the channel
    slack_client.api_call(
        "chat.postMessage",
        channel=channel,
        text=response or default_response
    )

